In [84]:
from selenium import webdriver
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium import webdriver
import pickle
from datetime import datetime

In [ ]:
#do only for first login! After, comment this part out
driver = webdriver.Chrome()
instagram = "https://www.instagram.com/"
driver.get(instagram)

Now login manually on your instagram account

In [ ]:
#Let's save the cookies 

In [82]:
def load_profiled_browser():
    options=webdriver.ChromeOptions()
    options.add_argument("user-data-dir=/home/nicolae/.config/google-chrome")
    #options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    #pickle.dump(driver.get_cookies(), open('cookies.pkl', 'wb'))
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    return driver 

In [83]:
instagram = "https://www.instagram.com/"
driver = load_profiled_browser()
time.sleep(3)
driver.get(instagram)

In [79]:
stories_remaining = True
try:
    own_story = driver.find_element_by_xpath('/html/body/div[1]/section/main/section/div[3]/div[1]/div/div[1]/span/img')
    own_story.click()
except NoSuchElementException: 
    try:       #nested, I know, but following EAFP principle
        own_story = driver.find_element_by_xpath('/html/body/div[1]/section/main/section/div[1]/div/div/div/div[1]/button/div[1]/span/img') 
        own_story.click()
    except NoSuchElementException:
        stories_remaining = False 
        print("no such element")

viewers = []        
next_story_button = 1
while stories_remaining:     #use Webdriverwait!
    try:
        view_count = driver.find_element_by_xpath('/html/body/div[1]/section/div/div/section/div[2]/div[3]/div[2]/button/div/span').text
        view_count = int(view_count.split("Vista por")[1])
        viewers.append(view_count)
        next_story = driver.find_element_by_xpath(f'/html/body/div[1]/section/div/div/section/div[2]/button[{next_story_button}]/div')
        next_story_button = 2
        time.sleep(1)
        next_story.click()
    except (NoSuchElementException, StaleElementReferenceException) as e:
        stories_remaining = False
viewers

[103, 98, 94]

In [40]:
pop_up

'CDwindow-1EBA02C78BC8C79238465531396DC1AA'

In [43]:
driver.window_handles

['CDwindow-1EBA02C78BC8C79238465531396DC1AA']